In [2121]:
import pandas as pd
import os
import numpy as np

In [2122]:
data_file = "life_expectancy.csv"
data_file2 = "uszips.csv"
data_file3 = "tax_rates.csv"
data_file4 = "merge_df.csv"
data_file5 = "education_rank.csv"
data_file6 = "prop_tax_rank.csv"

life_df = pd.read_csv(data_file)
zips = pd.read_csv(data_file2)
tax_rates = pd.read_csv(data_file3)
salary_house = pd.read_csv(data_file4)
ed_rank = pd.read_csv(data_file5)
prop_tax_rank = pd.read_csv(data_file6)
zips.sample(15)

,zip,lat,lng,city,state_id,state_name,zcta,parent_zcta,population,density,county_fips,county_name,county_weights,county_names_all,county_fips_all,imprecise,military,timezone
21849,62347,39.86648,-91.08905,Liberty,IL,Illinois,True,NaN,1793.0,5.9,17001,Adams,"{""17001"": 100}",Adams,17001,False,False,America/Chicago
33068,98258,48.03252,-122.07762,Lake Stevens,WA,Washington,True,NaN,51374.0,535.3,53061,Snohomish,"{""53061"": 100}",Snohomish,53061,False,False,America/Los_Angeles
14680,44612,40.63488,-81.46098,Bolivar,OH,Ohio,True,NaN,5516.0,80.4,39157,Tuscarawas,"{""39157"": 83.73, ""39151"": 16.27}",Tuscarawas|Stark,39157|39151,False,False,America/New_York
8527,27201,36.03258,-79.48418,Alamance,NC,North Carolina,True,NaN,101.0,178.4,37001,Alamance,"{""37001"": 100}",Alamance,37001,False,False,America/New_York
10167,30814,33.43604,-82.31879,Harlem,GA,Georgia,True,NaN,11201.0,80.8,13073,Columbia,"{""13073"": 83.95, ""13189"": 16.05}",Columbia|McDuffie,13073|13189,False,False,America/New_York
8066,25560,38.44736,-81.89908,Scott Depot,WV,West Virginia,True,NaN,7374.0,134.2,54079,Putnam,"{""54079"": 100}",Putnam,54079,False,False,America/New_York
13621,41267,37.87514,-82.43591,Warfield,KY,Kentucky,True,NaN,1529.0,98.9,21159,Martin,"{""21159"": 100}",Martin,21159,False,False,America/New_York
23489,66769,37.85317,-94.89379,Redfield,KS,Kansas,True,NaN,556.0,4.3,20011,Bourbon,"{""20011"": 100}",Bourbon,20011,False,False,America/Chicago
16353,48505,43.06948,-83.69079,Flint,MI,Michigan,True,NaN,18618.0,576.3,26049,Genesee,"{""26049"": 100}",Genesee,26049,False,False,America/Detroit
29127,83336,42.56260,-113.81699,Heyburn,ID,Idaho,True,NaN,6083.0,66.7,16067,Minidoka,"{""16067"": 100}",Minidoka,16067,False,False,America/Boise


In [2123]:
#cut down zips dataframe to only columns we need "county_name" and "zip"
zips = zips[["county_name", "zip", "state_id"]]
tax_rates = tax_rates[["ZipCode","EstimatedCombinedRate", "State"]]
#remove blanks from tax_rates dataframe
tax_rates = tax_rates.dropna()
#in county_names_all only keep the county names before the | delimiter
zips["county_name"] = zips["county_name"].str.split("|").str[0]
#rename count_name to "County" in zips dataframe
zips = zips.rename(columns={"county_name": "County", "state_id": "State"})
tax_rates = tax_rates.rename(columns={"ZipCode": "zip", "EstimatedCombinedRate": "Tax Rate"})

tax_rates.head()

,zip,Tax Rate,State
0,99501,0.0,AK
1,99502,0.0,AK
2,99503,0.0,AK
3,99504,0.0,AK
4,99505,0.0,AK


In [2124]:
zips.sample(10)

,County,zip,State
32472,Honolulu,96789,HI
25724,McClain,73031,OK
13741,Perry,41754,KY
31307,Kern,93206,CA
7312,Virginia Beach,23461,VA
8136,Raleigh,25836,WV
21496,Marshall,61540,IL
6355,Prince William,20143,VA
31101,San Bernardino,92342,CA
4553,Washington,15312,PA


In [2125]:
#create new column in tax_rates and start at 1 to end of column
tax_zip = pd.merge(tax_rates, zips, on=["zip", "State"], how="outer")
#group tax_zip by county and state and take the mean of the tax rate
stackranktax = tax_zip.groupby(["County", "State"]).mean()

stackranktax2 = stackranktax.sort_values("Tax Rate", ascending=False)
stackranktax2["Tax Rank"] = range(1, len(stackranktax2) + 1)
#turn county from index to a column
stackranktax2 = stackranktax2.reset_index()
# stackranktax2.sample(10)
stackranktax2 = stackranktax2.sort_values("County", ascending=True)
#rename State to state in stackranktax2
stackranktax2 = stackranktax2.rename(columns={"State": "state"})
stackranktax2.sample(10)

,County,state,zip,Tax Rate,Tax Rank
515,Fulton,GA,30266.657895,0.085382,516
2780,Dundy,NE,69032.250000,0.058750,2781
357,St. Tammany,LA,70448.875000,0.091319,358
1297,Sherman,TX,77376.666667,0.074167,1298
2881,Hitchcock,NE,69037.000000,0.055000,2882
1535,Coleman,TX,77454.111111,0.071389,1536
533,Grant,KS,67880.000000,0.085000,534
983,Garfield,CO,81633.666667,0.079200,984
612,Johnson,TX,76050.000000,0.082500,613
2927,Kewaunee,WI,54209.750000,0.055000,2928


In [2126]:
stackranktax2 = stackranktax2[["County", "state", "Tax Rank", "Tax Rate"]]
stackranktax2 = stackranktax2.reset_index()
stackranktax2.head()

,index,County,state,Tax Rank,Tax Rate
0,1357,Abbeville,SC,1358,0.073333
1,275,Acadia,LA,276,0.094542
2,2975,Accomack,VA,2976,0.053303
3,2470,Ada,ID,2471,0.060000
4,1751,Adair,IA,1752,0.070000


In [2127]:

#remove data with missing values
life_df = life_df.dropna()
life_df["state"] = life_df["County"].str.split(",").str[1]
county_df = life_df.groupby("County").mean()
county_df.head()

C:\Users\geoff\AppData\Local\Temp\ipykernel_19640\4238707294.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  county_df = life_df.groupby("County").mean()


,Census Tract Number,Life Expectancy,Life Expectancy Standard Error
County,,,
"Abbeville County, SC",9503.500000,77.650000,1.839250
"Acadia Parish, LA",9606.545455,74.445455,1.404873
"Accomack County, VA",904.500000,76.887500,2.098950
"Ada County, ID",42.513729,79.915254,1.651675
"Adair County, IA",9602.000000,78.133333,1.447033


In [2128]:


#sort by life expectancy
county_df = county_df.sort_values("Life Expectancy", ascending=False)
#add ranking column starting with 1 to end of dataframe
county_df["Rank"] = range(1, len(county_df) + 1)
county_df = county_df.reset_index()
county_df = county_df.sort_values("County", ascending=True)
#rename rank to "Life Expectancy Rank"
county_df = county_df.rename(columns={"Rank": "Life Expectancy Rank"})
county_df = county_df.reset_index()

county_df = county_df.sort_values("County", ascending=False)
county_df = county_df.reset_index()
stackranktax2.head()


,index,County,state,Tax Rank,Tax Rate
0,1357,Abbeville,SC,1358,0.073333
1,275,Acadia,LA,276,0.094542
2,2975,Accomack,VA,2976,0.053303
3,2470,Ada,ID,2471,0.060000
4,1751,Adair,IA,1752,0.070000


In [2129]:
stackranktax2["County"] = stackranktax2["County"]+ " County " + stackranktax2["state"]
county_df = county_df[['County', 'Life Expectancy Rank', 'Life Expectancy']]
#remove the ","" from the end of the county name in county_df
county_df["County"] = county_df["County"].str.split(",").str[0] + county_df["County"].str.split(",").str[1]

In [2130]:
county_df.head()

,County,Life Expectancy Rank,Life Expectancy
0,Ziebach County SD,2570,75.200000
1,Zavala County TX,1622,77.575000
2,Zapata County TX,1057,78.766667
3,Yuma County CO,723,79.650000
4,Yuma County AZ,1364,78.115217


In [2131]:
stackranktax2.head()

,index,County,state,Tax Rank,Tax Rate
0,1357,Abbeville County SC,SC,1358,0.073333
1,275,Acadia County LA,LA,276,0.094542
2,2975,Accomack County VA,VA,2976,0.053303
3,2470,Ada County ID,ID,2471,0.060000
4,1751,Adair County IA,IA,1752,0.070000


In [2132]:
#merge the two dataframes
new_df = pd.merge(county_df, stackranktax2, on="County")
new_df["overall rank"] = new_df["Life Expectancy Rank"] + new_df["Tax Rank"]
new_df = new_df.sort_values("overall rank", ascending=True)
#export to csv
new_df.sample(15)


,County,Life Expectancy Rank,Life Expectancy,index,state,Tax Rank,Tax Rate,overall rank
2311,Custer County MT,1982,76.766667,3157,MT,3158,0.000000,5140
1367,Livingston County MO,1479,77.875000,2222,MO,2223,0.063679,3702
2809,Bennington County VT,121,82.133333,2260,VT,2261,0.062941,2382
728,Queens County NY,212,81.483710,422,NY,423,0.088750,635
254,Upton County TX,508,80.250000,1255,TX,1256,0.075000,1764
802,Plymouth County IA,369,80.750000,1786,IA,1787,0.070000,2156
1285,Maricopa County AZ,1149,78.589182,521,AZ,522,0.085251,1671
2630,Camden County NC,294,81.050000,2103,NC,2104,0.067500,2398
1976,Gloucester County VA,1545,77.737500,2252,VA,2253,0.063000,3798
2348,Crawford County IN,2092,76.500000,1648,IN,1649,0.070000,3741


In [2133]:
salary_house.head()

,RegionName,State,Median_Household_Income_2020,1/31/2020,Ratio_Income_to_House,Rank
0,Hodgeman County,KS,60414,26757.84736,2.257805,1
1,Clark County,KS,58365,26291.68662,2.219903,2
2,Wallace County,KS,57386,26656.45185,2.152800,3
3,Stafford County,KS,55004,26052.60787,2.111267,4
4,Rawlins County,KS,54417,29023.74176,1.874913,5


In [2134]:
salary_house["RegionName"] = salary_house["RegionName"] +" " + salary_house["State"]
salary_house.sample(10)

,RegionName,State,Median_Household_Income_2020,1/31/2020,Ratio_Income_to_House,Rank
581,Howard County AR,AR,40566,70609.21827,0.574514,582
617,Box Butte County NE,NE,62621,110829.92250,0.565019,618
86,Mason County IL,IL,53503,52127.80199,1.026381,87
753,Pike County AR,AR,43133,80963.88997,0.532744,754
2701,Mason County WA,WA,63785,284158.77510,0.224470,2702
1802,Crowley County CO,CO,36836,99893.95015,0.368751,1803
1273,Lafayette County MO,MO,62043,141445.14260,0.438636,1274
1539,Daviess County KY,KY,54948,136283.42280,0.403189,1540
1531,Beadle County SD,SD,54033,133781.25940,0.403891,1532
1401,Aiken County SC,SC,57128,135727.53080,0.420902,1402


In [2135]:
new_df = new_df.reset_index()
new_df = new_df[["County", "Life Expectancy Rank", "Life Expectancy", "Tax Rank", "Tax Rate"]]
new_df.head()

,County,Life Expectancy Rank,Life Expectancy,Tax Rank,Tax Rate
0,San Mateo County CA,67,82.739474,209,0.096750
1,Garfield County WA,86,82.500000,263,0.095000
2,King County WA,261,81.201852,112,0.099709
3,Ouray County CO,58,83.200000,323,0.092500
4,Santa Clara County CA,75,82.653125,308,0.093125


In [2136]:


salary_house = salary_house.rename(columns={"RegionName": "County", "State": "state", "Rank": "Salary/House Rank", "Ratio_Income_to_House": "salary/house"})

salary_house = salary_house[["County", "Salary/House Rank", "salary/house"]]
salary_house.head()

#merged_df = merged_df.merge(salary_house, on="County")


,County,Salary/House Rank,salary/house
0,Hodgeman County KS,1,2.257805
1,Clark County KS,2,2.219903
2,Wallace County KS,3,2.152800
3,Stafford County KS,4,2.111267
4,Rawlins County KS,5,1.874913


In [2137]:
overall_df = pd.merge(new_df, salary_house, on=["County"])
overall_df.head()

,County,Life Expectancy Rank,Life Expectancy,Tax Rank,Tax Rate,Salary/House Rank,salary/house
0,San Mateo County CA,67,82.739474,209,0.096750,2917,0.103167
1,Garfield County WA,86,82.500000,263,0.095000,1576,0.399438
2,King County WA,261,81.201852,112,0.099709,2871,0.165247
3,Ouray County CO,58,83.200000,323,0.092500,2864,0.167261
4,Santa Clara County CA,75,82.653125,308,0.093125,2910,0.124518


In [2138]:
prop_tax_rank.head(25)


,state,effective tax rate (2023),Rank
0,HI,0.29%,1.0
1,AL,0.41%,2.0
2,CO,0.51%,3.0
3,NV,0.55%,4.0
4,SC,0.56%,5.5
5,LA,0.56%,5.5
6,UT,0.58%,7.5
7,DE,0.58%,7.5
8,WV,0.59%,9.0
9,WY,0.61%,10.0


In [2139]:
#add state to overall_df by using string split 2 of County column
overall_df["state"] = overall_df["County"].str.split(" ").str[2]
#for each state, add the value in prop_tax_rank to the overall_df
overall_df = overall_df.merge(prop_tax_rank, on="state")
weight_prop_tax = overall_df[["County", "Rank", "state"]]
#remove effective tax rate (2023) column
weight_prop_tax.sample(10)


,County,Rank,state
1949,Beadle County SD,34.0,SD
1338,Mahoning County OH,39.0,OH
250,Rawlins County KS,36.0,KS
551,Limestone County TX,45.0,TX
1404,Bonner County ID,13.0,ID
660,Knox County IL,49.0,IL
1856,Pulaski County IN,20.5,IN
203,Amador County CA,15.0,CA
2667,Clark County KY,22.0,KY
1006,Utah County UT,7.5,UT


In [2140]:
#sort by Rank
weight_prop_tax = weight_prop_tax.sort_values("Rank", ascending=True)
#for each county in each state find the number of counties in that state
weight_prop_tax["count"] = weight_prop_tax.groupby("state")["state"].transform("count")
#for each row find where the state changes, then add the count to the weighted rank

#change "count" to a float
weight_prop_tax["count"] = weight_prop_tax["count"].astype(float)

weight_prop_tax["rank property tax"] = range(1, len(weight_prop_tax) + 1)
weight_prop_tax.head(10)
#get average "rank weight" for each state
weight_state = weight_prop_tax.groupby("state").mean()
weight_state = weight_state[['rank property tax']]
weight_state.head(20)

C:\Users\geoff\AppData\Local\Temp\ipykernel_19640\1239556251.py:13: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  weight_state = weight_prop_tax.groupby("state").mean()


,rank property tax
state,
AL,35.500000
AR,321.000000
AZ,321.000000
CA,516.500000
CO,93.000000
CT,2600.500000
DE,201.500000
FL,1105.000000
GA,1271.025974


In [2141]:
#add the rank weight to the overall_df for each state
overall_df = overall_df.merge(weight_state, on="state")
overall_df.head(10)


,County,Life Expectancy Rank,Life Expectancy,Tax Rank,Tax Rate,Salary/House Rank,salary/house,state,effective tax rate (2023),Rank,rank property tax
0,Garfield County WA,86,82.500000,263,0.095000,1576,0.399438,WA,0.94%,27.0,1417.0
1,King County WA,261,81.201852,112,0.099709,2871,0.165247,WA,0.94%,27.0,1417.0
2,Jefferson County WA,229,81.371429,364,0.091000,2854,0.173072,WA,0.94%,27.0,1417.0
3,Island County WA,215,81.436842,428,0.088500,2827,0.185468,WA,0.94%,27.0,1417.0
4,Snohomish County WA,748,79.591096,122,0.099185,2817,0.187724,WA,0.94%,27.0,1417.0
5,Whatcom County WA,432,80.538710,476,0.086750,2857,0.169866,WA,0.94%,27.0,1417.0
6,Douglas County WA,284,81.100000,672,0.082444,2835,0.182357,WA,0.94%,27.0,1417.0
7,Okanogan County WA,435,80.520000,559,0.083895,2553,0.258847,WA,0.94%,27.0,1417.0
8,Chelan County WA,460,80.423077,575,0.083538,2829,0.184754,WA,0.94%,27.0,1417.0
9,Lincoln County WA,295,81.050000,759,0.080417,1994,0.345556,WA,0.94%,27.0,1417.0


In [2142]:
#take the first count value for each state and put in a new dataframe
weight_prop_tax2 = weight_prop_tax.groupby("state").first()
#keep only one state value and the count
weight_prop_tax2 = weight_prop_tax2[["count"]]
weight_prop_tax2
weight_prop_tax2["count"] = weight_prop_tax2["count"].astype(float)
weight_prop_tax2["count"] = weight_prop_tax2["count"] / 2
weight_prop_tax2.head()



,count
state,
AL,32.0
AR,35.0
AZ,6.5
CA,22.0
CO,25.5


In [2143]:
#change name of effective tax rate (2023)" to "property tax rate"
overall_df = overall_df.rename(columns={"effective tax rate (2023)": "property tax rate"})

In [2144]:
overall_df.head()

,County,Life Expectancy Rank,Life Expectancy,Tax Rank,Tax Rate,Salary/House Rank,salary/house,state,property tax rate,Rank,rank property tax
0,Garfield County WA,86,82.500000,263,0.095000,1576,0.399438,WA,0.94%,27.0,1417.0
1,King County WA,261,81.201852,112,0.099709,2871,0.165247,WA,0.94%,27.0,1417.0
2,Jefferson County WA,229,81.371429,364,0.091000,2854,0.173072,WA,0.94%,27.0,1417.0
3,Island County WA,215,81.436842,428,0.088500,2827,0.185468,WA,0.94%,27.0,1417.0
4,Snohomish County WA,748,79.591096,122,0.099185,2817,0.187724,WA,0.94%,27.0,1417.0


In [2145]:
ed_rank.head()

,State,Area name,hs_diploma_up,Rank
0,MT,Petroleum County,99.4,1.0
1,CO,Ouray County,98.6,2.0
2,CO,Custer County,98.4,3.0
3,AK,Denali Borough,98.3,4.5
4,CO,San Miguel County,98.3,4.5


In [2146]:
#rename Area name and add state to it
ed_rank["Area name"] = ed_rank["Area name"] + " " + ed_rank["State"]
ed_rank = ed_rank.rename(columns={"Area name": "County", "Rank": "Education Rank", "hs_diploma_up": "percent hs diploma"})
ed_rank.head()

,State,County,percent hs diploma,Education Rank
0,MT,Petroleum County MT,99.4,1.0
1,CO,Ouray County CO,98.6,2.0
2,CO,Custer County CO,98.4,3.0
3,AK,Denali Borough AK,98.3,4.5
4,CO,San Miguel County CO,98.3,4.5


In [2147]:
#remove state column from ed_rank
ed_rank = ed_rank[["County", "Education Rank", "percent hs diploma"]]

In [2148]:
#merge ed_rank with overall_df
overall_df = overall_df.merge(ed_rank, on="County")
#delete "Rank" from overall_df
overall_df = overall_df.drop(columns=["Rank"])
overall_df.head()

,County,Life Expectancy Rank,Life Expectancy,Tax Rank,Tax Rate,Salary/House Rank,salary/house,state,property tax rate,rank property tax,Education Rank,percent hs diploma
0,Garfield County WA,86,82.500000,263,0.095000,1576,0.399438,WA,0.94%,1417.0,355.5,94.0
1,King County WA,261,81.201852,112,0.099709,2871,0.165247,WA,0.94%,1417.0,454.0,93.6
2,Jefferson County WA,229,81.371429,364,0.091000,2854,0.173072,WA,0.94%,1417.0,69.0,96.1
3,Island County WA,215,81.436842,428,0.088500,2827,0.185468,WA,0.94%,1417.0,121.0,95.5
4,Snohomish County WA,748,79.591096,122,0.099185,2817,0.187724,WA,0.94%,1417.0,625.0,92.9


In [2149]:
#create a sum rank column and add up all the ranks, then sor by sum rank and give the overall rank
overall_df["sum rank"] = overall_df["Life Expectancy Rank"] + overall_df["Tax Rank"] + overall_df["Salary/House Rank"] + overall_df["rank property tax"]+ overall_df["Education Rank"]
overall_df = overall_df.sort_values("sum rank", ascending=True)
overall_df["overall rank"] = range(1, len(overall_df) + 1)
overall_df = overall_df.reset_index(drop=True)
rank_df = overall_df[["County", "state", "overall rank", "sum rank", "Life Expectancy Rank", "Tax Rank", "Salary/House Rank", "rank property tax"]]

overall_df.reset_index(drop=True)

overall_df.head(10)

,County,Life Expectancy Rank,Life Expectancy,Tax Rank,Tax Rate,Salary/House Rank,salary/house,state,property tax rate,rank property tax,Education Rank,percent hs diploma,sum rank,overall rank
0,Wallace County KS,190,81.6,585,0.083333,3,2.152800,KS,1.43%,1789.0,109.0,95.6,2676.0,1
1,Eureka County NV,226,81.4,2003,0.069333,471,0.620171,NV,0.55%,125.5,168.0,95.1,2993.5,2
2,Lincoln County KS,499,80.3,622,0.082500,10,1.696424,KS,1.43%,1789.0,121.0,95.5,3041.0,3
3,Lane County KS,653,79.8,531,0.085000,13,1.640908,KS,1.43%,1789.0,97.0,95.7,3083.0,4
4,Gove County KS,588,80.0,455,0.087500,17,1.584128,KS,1.43%,1789.0,295.0,94.3,3144.0,5
5,Greenlee County AZ,73,82.7,727,0.081000,205,0.797833,AZ,0.62%,321.0,1891.5,87.7,3217.5,6
6,Cheyenne County KS,222,81.4,384,0.090000,69,1.070692,KS,1.43%,1789.0,846.5,92.1,3310.5,7
7,Rawlins County KS,324,80.9,1084,0.077500,5,1.874913,KS,1.43%,1789.0,121.0,95.5,3323.0,8
8,Ouray County CO,58,83.2,323,0.092500,2864,0.167261,CO,0.51%,93.0,2.0,98.6,3340.0,9
9,Russell County KS,702,79.7,382,0.090000,18,1.535223,KS,1.43%,1789.0,571.5,93.1,3462.5,10


In [2150]:
#group by state and return the best county in each state
state_list = overall_df.groupby("state").first()
state_list = state_list.sort_values("overall rank", ascending=True)
state_list.head(50)

,County,Life Expectancy Rank,Life Expectancy,Tax Rank,Tax Rate,Salary/House Rank,salary/house,property tax rate,rank property tax,Education Rank,percent hs diploma,sum rank,overall rank
state,,,,,,,,,,,,,
KS,Wallace County KS,190,81.600000,585,0.083333,3,2.152800,1.43%,1789.000000,109.0,95.6,2676.000000,1
NV,Eureka County NV,226,81.400000,2003,0.069333,471,0.620171,0.55%,125.500000,168.0,95.1,2993.500000,2
AZ,Greenlee County AZ,73,82.700000,727,0.081000,205,0.797833,0.62%,321.000000,1891.5,87.7,3217.500000,6
CO,Ouray County CO,58,83.200000,323,0.092500,2864,0.167261,0.51%,93.000000,2.0,98.6,3340.000000,9
TN,Williamson County TN,260,81.231250,158,0.097500,2651,0.239308,0.66%,448.000000,97.0,95.7,3614.000000,14
WA,Garfield County WA,86,82.500000,263,0.095000,1576,0.399438,0.94%,1417.000000,355.5,94.0,3697.500000,18
OK,Washita County OK,594,79.975000,232,0.096000,222,0.781145,0.90%,1206.826667,1470.0,89.6,3724.826667,19
AL,Shelby County AL,1259,78.355556,298,0.093529,1729,0.377533,0.41%,35.500000,454.0,93.6,3775.500000,21
MO,Putnam County MO,56,83.300000,1531,0.071500,589,0.572758,0.98%,1487.000000,274.5,94.4,3937.500000,30


In [2151]:
#export state_list to csv
state_list.to_csv("state_list.csv")
#export overall_df to csv
overall_df.to_csv("overall_df.csv")